In [1]:
import pandas as pd
import os
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.metrics import mean_squared_error
import warnings
import pickle

In [8]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-0-exp')

2023/03/12 21:10:24 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/03/12 21:10:24 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/03/12 21:10:24 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-0-exp' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/bcthakredata/mlops_zoomcamp/week2/experiment-tracking/models/mlruns/2', creation_time=1678655424788, experiment_id='2', last_update_time=1678655424788, lifecycle_stage='active', name='nyc-taxi-0-exp', tags={}>

In [9]:
def read_parquet_file(filename):
    """
    To read the data frame from parquet file
    
    """
    
    df = pd.read_parquet(filename)
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df.duration.apply(lambda td : td.total_seconds()/60)
    
    df = df[((df.duration >= 1) & (df.duration <=60))]
    
    categorical = ['PULocationID','DOLocationID']
        
    df[categorical] = df[categorical].astype(str)
    
    return df

In [10]:
df_train = read_parquet_file('/home/bcthakredata/mlops_zoomcamp/week1/data/green_tripdata_2022-01.parquet')
df_val = read_parquet_file('/home/bcthakredata/mlops_zoomcamp/week1/data/green_tripdata_2022-02.parquet')

In [11]:
df_train['PU_DO_PAIR'] = df_train['PULocationID'] + "_" + df_train['DOLocationID']
df_val['PU_DO_PAIR'] = df_val['PULocationID'] + "_" + df_val['DOLocationID']

In [12]:
categorical = ['PULocationID','DOLocationID','PU_DO_PAIR']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
with mlflow.start_run():

    mlflow.set_tag("developer", "bt")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

FileNotFoundError: [Errno 2] No such file or directory: 'models/lin_reg.bin'